# 问题1.1
# 步骤 1：读取数据

从Excel和CSV文件中加载数据：
1. 男大学生的一日食谱 (`附件1：1名男大学生的一日食谱.xlsx`)
2. 女大学生的一日食谱 (`附件2：1名女大学生的一日食谱.xlsx`)
3. 高校学生食堂一日三餐主要食物信息统计表 (`附件3：某高校学生食堂一日三餐主要食物信息统计表.xlsx`)
4. 男大学生食谱 (`食谱.xlsx`，工作表 `男大`)
5. 女大学生食谱 (`食谱.xlsx`，工作表 `女大`)
6. 食物营养数据 (`foods_nutrients.csv`)




In [1]:
import pandas as pd
import numpy as np

df1 = pd.read_excel('附件1：1名男大学生的一日食谱.xlsx')
df2 = pd.read_excel('附件2：1名女大学生的一日食谱.xlsx')
df3 = pd.read_excel('附件3：某高校学生食堂一日三餐主要食物信息统计表.xlsx')
food_male = pd.read_excel('食谱.xlsx', sheet_name='男大')
food_female = pd.read_excel('食谱.xlsx', sheet_name='女大')


In [2]:
foods_nutrients = pd.read_csv('foods_nutrients.csv')


# 步骤 2：提取餐食信息

In [3]:

def extract_meals(df, sheet_name):
    foods = df[sheet_name].dropna().values
    meals = {}
    meal = []
    for item in foods:
        if item in ['早餐', '午餐', '晚餐']:
            key = item
            meal = []
        elif item != '食物名称':
            meal.append(item)
        else:
            meals[key] = meal
    return meals

meals_male = extract_meals(df1, '1名男大学生的一日食谱')
meals_female = extract_meals(df2, '1名女大学生的1日食谱')

# 步骤 3：前向填充缺失值

**计算公式**：
对于每个缺失值，用前一个非缺失值进行填充：
$$
\text{value}_{\text{current}} = \text{value}_{\text{previous}} \quad \text{if value}_{\text{current}} \text{ is null}
$$


In [4]:
food_male = pd.read_excel('食谱.xlsx', sheet_name='男大').fillna(0)
food_female = pd.read_excel('食谱.xlsx', sheet_name='女大').fillna(method='ffill')

# 步骤 4：计算营养素总和

**计算公式**：
定义总营养素初始值为0：
$$
\text{total\_nutrients} = \{ \text{热量 (kcal)}: 0, \text{蛋白质 (g)}: 0, \text{脂肪 (g)}: 0, \text{碳水化合物 (g)}: 0 \}
$$

对于每种食物，计算可食部克数乘以每份数并除以100：
$$
\text{portion\_size} = \frac{\text{可食部（克/份）} \times \text{食用份数}}{100}
$$

然后计算每种营养素的总摄入量：
$$
\text{total\_nutrients}[\text{热量 (kcal)}] += \text{nutrient}[\text{热量 (kcal)}] \times \text{portion\_size}
$$
$$
\text{total\_nutrients}[\text{蛋白质 (g)}] += \text{nutrient}[\text{蛋白质 (g)}] \times \text{portion\_size}
$$
$$
\text{total\_nutrients}[\text{脂肪 (g)}] += \text{nutrient}[\text{脂肪 (g)}] \times \text{portion\_size}
$$
$$
\text{total\_nutrients}[\text{碳水化合物 (g)}] += \text{nutrient}[\text{碳水化合物 (g)}] \times \text{portion\_size}
$$

对于一天中的所有餐，累计每餐的总营养素：
$$
\text{whole\_day\_nutrients}[\text{热量 (kcal)}] += \text{total\_nutrients}[\text{热量 (kcal)}]
$$
$$
\text{whole\_day\_nutrients}[\text{蛋白质 (g)}] += \text{total\_nutrients}[\text{蛋白质 (g)}]
$$
$$
\text{whole\_day\_nutrients}[\text{脂肪 (g)}] += \text{total\_nutrients}[\text{脂肪 (g)}]
$$
$$
\text{whole\_day\_nutrients}[\text{碳水化合物 (g)}] += \text{total\_nutrients}[\text{碳水化合物 (g)}]
$$


In [5]:
food_female_unique = food_female.drop_duplicates(subset=['主要成分'])['主要成分'].values
food_male_unique = food_male.drop_duplicates(subset=['主要成分'])['主要成分'].values

# 读取食物营养素数据
foods_nutrients = pd.read_csv('foods_nutrients.csv')

# 计算每餐和一整天的总营养素
def calculate_nutrients(meals, food_data):
    total_day_nutrients = {'热量 (kcal)': 0, '蛋白质 (g)': 0, '脂肪 (g)': 0, '碳水化合物 (g)': 0}
    daily_nutrients_list = []

    for meal, items in meals.items():
        meal_nutrients = {'热量 (kcal)': 0, '蛋白质 (g)': 0, '脂肪 (g)': 0, '碳水化合物 (g)': 0}
        for item in items:
            for _, food in food_data.iterrows():
                if food['食物名称'] == item:
                    nutrient = foods_nutrients[foods_nutrients['食物'] == food['主要成分']].iloc[0]
                    portion_size = food['可食部（克/份）'] * food['食用份数'] / 100
                    meal_nutrients['热量 (kcal)'] += nutrient['热量 (kcal)'] * portion_size
                    meal_nutrients['蛋白质 (g)'] += nutrient['蛋白质 (g)'] * portion_size
                    meal_nutrients['脂肪 (g)'] += nutrient['脂肪 (g)'] * portion_size
                    meal_nutrients['碳水化合物 (g)'] += nutrient['碳水化合物 (g)'] * portion_size
        daily_nutrients_list.append((meal, meal_nutrients))
        total_day_nutrients['热量 (kcal)'] += meal_nutrients['热量 (kcal)']
        total_day_nutrients['蛋白质 (g)'] += meal_nutrients['蛋白质 (g)']
        total_day_nutrients['脂肪 (g)'] += meal_nutrients['脂肪 (g)']
        total_day_nutrients['碳水化合物 (g)'] += meal_nutrients['碳水化合物 (g)']

    return total_day_nutrients, daily_nutrients_list

# 计算男大学生和女大学生的一天总营养素
whole_day_nutrients_male, male_ls = calculate_nutrients(meals_male, food_male)
whole_day_nutrients_female, female_ls = calculate_nutrients(meals_female, food_female)

print("男大学生每餐的营养素：")
for meal, nutrients in male_ls:
    print(f"{meal}：{nutrients}")

print("\n男大学生一天的总营养素：", whole_day_nutrients_male)

print("\n女大学生每餐的营养素：")
for meal, nutrients in female_ls:
    print(f"{meal}：{nutrients}")

print("\n女大学生一天的总营养素：", whole_day_nutrients_female)

男大学生每餐的营养素：
早餐：{'热量 (kcal)': 686.5, '蛋白质 (g)': 27.9, '脂肪 (g)': 6.875, '碳水化合物 (g)': 129.85}
午餐：{'热量 (kcal)': 971.6, '蛋白质 (g)': 24.48, '脂肪 (g)': 30.78, '碳水化合物 (g)': 144.2}
晚餐：{'热量 (kcal)': 1031.0, '蛋白质 (g)': 34.08, '脂肪 (g)': 53.15, '碳水化合物 (g)': 112.975}

男大学生一天的总营养素： {'热量 (kcal)': 2689.1, '蛋白质 (g)': 86.46, '脂肪 (g)': 90.805, '碳水化合物 (g)': 387.025}

女大学生每餐的营养素：
早餐：{'热量 (kcal)': 246.6, '蛋白质 (g)': 18.92, '脂肪 (g)': 6.24, '碳水化合物 (g)': 33.76}
午餐：{'热量 (kcal)': 585.6, '蛋白质 (g)': 18.139999999999997, '脂肪 (g)': 30.470000000000002, '碳水化合物 (g)': 58.065000000000005}
晚餐：{'热量 (kcal)': 461.15, '蛋白质 (g)': 17.235, '脂肪 (g)': 13.51, '碳水化合物 (g)': 66.815}

女大学生一天的总营养素： {'热量 (kcal)': 1293.35, '蛋白质 (g)': 54.295, '脂肪 (g)': 50.22, '碳水化合物 (g)': 158.64}


# 步骤 5：计算膳食的标准参考值

**计算公式**：
设定男生每日营养素推荐摄入量，定义为：
$$
\text{reference\_intake\_male} = \{ \text{热量 (kcal)}: 2400, \text{蛋白质 (g)}: 60, \text{脂肪 (g)}: 70, \text{碳水化合物 (g)}: 300 \}
$$

设定女生每日营养素推荐摄入量，定义为：
$$
\text{reference\_intake\_female} = \{ \text{热量 (kcal)}: 1900, \text{蛋白质 (g)}: 50, \text{脂肪 (g)}: 60, \text{碳水化合物 (g)}: 250 \}
$$

# 步骤 6：标准化膳食数据

**计算公式**：
标准化处理实际膳食数据和参考膳食数据：
$$
X_i = \frac{x_i}{x_{\max}}
$$
其中，$X_i$为标准化后的值，$x_i$为实际膳食数据，$x_{\max}$为该营养素的最大值。


In [6]:
# 计算膳食的标准参考值
reference_intake_male = {'热量 (kcal)': 2400, '蛋白质 (g)': 60, '脂肪 (g)': 70, '碳水化合物 (g)': 300}
reference_intake_female = {'热量 (kcal)': 1900, '蛋白质 (g)': 50, '脂肪 (g)': 60, '碳水化合物 (g)': 250}

# 标准化膳食数据
def standardize_data(actual_intake, reference_intake):
    standardized_data = {}
    for nutrient in actual_intake:
        standardized_data[nutrient] = actual_intake[nutrient] / reference_intake[nutrient]
    return standardized_data

male_standardized = standardize_data(whole_day_nutrients_male, reference_intake_male)
female_standardized = standardize_data(whole_day_nutrients_female, reference_intake_female)



# 步骤 7：计算关联度

**计算公式**：
计算每个营养素的灰色关联度，定义为：
$$
\gamma_i = \frac{\min \Delta + \rho \cdot \max \Delta}{\Delta_i + \rho \cdot \max \Delta}
$$
其中，$\Delta_i$为实际膳食与参考膳食之间的差异值，$\rho$为分辨系数，通常取值为0.5。

# 步骤 8：综合关联度评价

**计算公式**：
综合关联度评价，定义为：
$$
\Gamma = \frac{1}{n} \sum_{i=1}^{n} \gamma_i
$$
其中，$\gamma_i$为每个营养素的关联度，$n$为营养素的总数。

In [7]:
# 计算灰色关联度
def calculate_grey_relation(actual, reference, rho=0.5):
    delta = {nutrient: abs(actual[nutrient] - reference[nutrient]) for nutrient in actual}
    delta_min = min(delta.values())
    delta_max = max(delta.values())
    grey_relation = {nutrient: (delta_min + rho * delta_max) / (delta[nutrient] + rho * delta_max) for nutrient in delta}
    return grey_relation

male_grey_relation = calculate_grey_relation(male_standardized, {k: 1 for k in reference_intake_male})
female_grey_relation = calculate_grey_relation(female_standardized, {k: 1 for k in reference_intake_female})

# 综合关联度评价
def comprehensive_evaluation(grey_relation):
    return np.mean(list(grey_relation.values()))

male_comprehensive_score = comprehensive_evaluation(male_grey_relation)
female_comprehensive_score = comprehensive_evaluation(female_grey_relation)

print(f"男大学生膳食的综合灰色关联度评分：{male_comprehensive_score}")
print(f"女大学生膳食的综合灰色关联度评分：{female_comprehensive_score}")


男大学生膳食的综合灰色关联度评分：0.7104495061764925
女大学生膳食的综合灰色关联度评分：0.7005290163096832


总体过程：

读取数据：从Excel和CSV文件中加载所有需要的数据。

提取餐食信息：从数据集中提取男大学生和女大学生的一日三餐信息。

前向填充缺失值：使用前向填充方法填充食谱数据中的缺失值。

计算营养素总和：计算每餐和全天的总营养素。

计算膳食的标准参考值：根据推荐摄入量设定男生和女生的膳食标准参考值。

标准化膳食数据：将实际摄入的营养素数据标准化。

计算灰色关联度：计算每个营养素的灰色关联度。

综合关联度评价：计算综合关联度评分，对膳食进行全面评价。

# 问题1.2

### 确定营养素缺乏和超出推荐值的部分

#### 步骤名称
1. 确定营养素缺乏和超出部分
2. 计算缺乏的营养素
3. 计算超出的营养素

#### 数学公式模型

1. **确定营养素缺乏和超出部分**

对于每种营养素 $ N $，计算实际摄入量 $ A_N $ 和推荐摄入量 $ R_N $ 之间的差异。

2. **计算缺乏的营养素**

如果实际摄入量小于推荐摄入量，则计算缺乏的营养素：
$$ D_N = R_N - A_N $$
其中，$ D_N $ 表示缺乏的营养素量。

3. **计算超出的营养素**

如果实际摄入量大于推荐摄入量，则计算超出的营养素：
$$ E_N = A_N - R_N $$
其中，$ E_N $ 表示超出的营养素量。

#### 输出结果

根据上述计算结果，输出男大学生和女大学生膳食中营养素的缺乏和超出部分。


#### 男大学生膳食的营养素超出
$$
\text{male_excesses} = \{ N : A_N - R_N \}
$$

#### 女大学生膳食的营养素缺乏
$$
\text{female_deficiencies} = \{ N : R_N - A_N \}
$$



In [8]:
# 确定营养素缺乏和超出推荐值的部分
def identify_deficiencies_and_excesses(actual_intake, reference_intake):
    deficiencies = {}
    excesses = {}
    for nutrient in actual_intake:
        if actual_intake[nutrient] < reference_intake[nutrient]:
            deficiencies[nutrient] = reference_intake[nutrient] - actual_intake[nutrient]
        elif actual_intake[nutrient] > reference_intake[nutrient]:
            excesses[nutrient] = actual_intake[nutrient] - reference_intake[nutrient]
    return deficiencies, excesses

male_deficiencies, male_excesses = identify_deficiencies_and_excesses(whole_day_nutrients_male, reference_intake_male)
female_deficiencies, female_excesses = identify_deficiencies_and_excesses(whole_day_nutrients_female, reference_intake_female)

print(f"\n男大学生膳食的营养素缺乏：{male_deficiencies}")
print(f"男大学生膳食的营养素超出：{male_excesses}")
print(f"\n女大学生膳食的营养素缺乏：{female_deficiencies}")
print(f"女大学生膳食的营养素超出：{female_excesses}")





男大学生膳食的营养素缺乏：{}
男大学生膳食的营养素超出：{'热量 (kcal)': 289.0999999999999, '蛋白质 (g)': 26.459999999999994, '脂肪 (g)': 20.805000000000007, '碳水化合物 (g)': 87.02499999999998}

女大学生膳食的营养素缺乏：{'热量 (kcal)': 606.6500000000001, '脂肪 (g)': 9.780000000000001, '碳水化合物 (g)': 91.36000000000001}
女大学生膳食的营养素超出：{'蛋白质 (g)': 4.295000000000002}


In [9]:
df3_breakfast = pd.read_excel('附件3.xlsx', sheet_name='早餐')
df3_lunch = pd.read_excel('附件3.xlsx', sheet_name='午餐')
df3_dinner = pd.read_excel('附件3.xlsx', sheet_name='晚餐')
df3_dinner 

,序号,食物名称,主要成分,食物编码,可食部\n（克/份）,价格\n（元/份）,是否\n可半份
0,1.0,大米饭,稻米,012001x,25,0.5,否
1,2.0,馒头,小麦粉,011201x,50,1.0,否
2,3.0,花卷,小麦粉,011201x,50,1.0,否
3,4.0,豆浆,黄豆,31101,10,1.5,否
4,5.0,小米粥,小米,15101,15,0.5,否
...,...,...,...,...,...,...,...
131,45.0,西瓜,西瓜,066201x,100,1.0,否
132,46.0,香蕉,香蕉,65033,100,1.0,否
133,47.0,柚子,柚子,64301,100,1.0,否
134,48.0,苹果,苹果,061101x,100,1.0,否


### 膳食调整策略

#### 数学公式

1. **计算营养素总和**
   对于每餐和一整天的营养素总和进行计算：

   $$
   T_{day} = \sum_{i=1}^{n} M_{i}
   $$

   其中，$ T_{day} $ 表示一整天的营养素总和，$ M_{i} $ 表示每餐的营养素总和。

2. **选择最合理的调整建议并修改膳食**

   根据调整建议修改膳食，处理减少和增加的部分：

   $$
   M_{updated} = M_{original} - \text{减少部分} + \text{增加部分}
   $$

3. **标准化膳食数据**

   对实际摄入的营养素数据进行标准化处理：

   $$
   S_{N} = \frac{A_{N}}{R_{N}}
   $$

   其中，$ S_{N} $ 表示标准化后的营养素，$ A_{N} $ 表示实际摄入量，$ R_{N} $ 表示推荐摄入量。

4. **计算灰色关联度**

   计算实际摄入量与推荐摄入量之间的灰色关联度：

   $$
   \Delta_{N} = |A_{N} - R_{N}|
   $$

   其中，$ \Delta_{N} $ 表示实际摄入量与推荐摄入量的差值。

   $$
   \xi_{N} = \frac{\Delta_{min} + \rho \Delta_{max}}{\Delta_{N} + \rho \Delta_{max}}
   $$

   其中，$ \xi_{N} $ 表示灰色关联度，$ \Delta_{min} $ 表示所有营养素差值中的最小值，$ \Delta_{max} $ 表示所有营养素差值中的最大值，$ \rho $ 为分辨系数，取值范围为 0 到 1。

5. **综合关联度评价**

   计算所有营养素的灰色关联度的平均值，作为综合评价结果：

   $$
   \Gamma = \frac{1}{n} \sum_{i=1}^{n} \xi_{i}
   $$

   其中，$ \Gamma $ 表示综合灰色关联度，$ \xi_{i} $ 表示第 $ i $ 个营养素的灰色关联度，$ n $ 表示营养素的数量。

6. **生成减少和增加食物的所有组合**

   生成减少和增加食物的所有可能组合：

   $$
   C_{reduce} = \text{itertools.combinations}(R, k)
   $$

   $$
   C_{add} = \text{itertools.combinations}(A, m)
   $$

   其中，$ C_{reduce} $ 表示减少食物的组合，$ R $ 表示可减少的食物集合，$ k $ 表示减少的食物数量，$ C_{add} $ 表示增加食物的组合，$ A $ 表示可增加的食物集合，$ m $ 表示增加的食物数量。

7. **定义食物调整策略**

   根据缺乏和超出部分生成增加和减少的食物调整策略：

   $$
   \text{adjustments} = \{\text{增加}: \{\}, \text{减少}: \{\}\}
   $$

8. **遍历所有组合，找到最佳的膳食调整方案**

   通过遍历所有可能的减少和增加食物的组合，找到使灰色关联度最高的调整方案：

   $$
   \text{best\_score} = \max(\Gamma_{total})
   $$

   其中，$ \Gamma_{total} $ 表示所有组合的综合灰色关联度。

9. **更新膳食并重新计算营养素总和**

   根据最佳调整方案更新膳食，并重新计算更新后的营养素总和：

   $$
   M_{updated} = M_{original} - \text{减少部分} + \text{增加部分}
   $$

10. **打印调整后的膳食建议和新的灰色关联度评分**

    输出调整后的膳食策略和新的综合灰色关联度评分：

    $$
    \text{print}(\Gamma_{updated})
    $$


In [10]:
import itertools
import pandas as pd
import numpy as np

def calculate_nutrients(meals, food_data):
    total_day_nutrients = {'热量 (kcal)': 0, '蛋白质 (g)': 0, '脂肪 (g)': 0, '碳水化合物 (g)': 0}
    daily_nutrients_list = []

    for meal, items in meals.items():
        meal_nutrients = {'热量 (kcal)': 0, '蛋白质 (g)': 0, '脂肪 (g)': 0, '碳水化合物 (g)': 0}
        for item in items:
            for _, food in food_data.iterrows():
                if food['食物名称'] == item:
                    nutrient = foods_nutrients[foods_nutrients['食物'] == food['主要成分']].iloc[0]
                    portion_size = food['可食部（克/份）'] * food['食用份数'] / 100
                    meal_nutrients['热量 (kcal)'] += nutrient['热量 (kcal)'] * portion_size
                    meal_nutrients['蛋白质 (g)'] += nutrient['蛋白质 (g)'] * portion_size
                    meal_nutrients['脂肪 (g)'] += nutrient['脂肪 (g)'] * portion_size
                    meal_nutrients['碳水化合物 (g)'] += nutrient['碳水化合物 (g)'] * portion_size
        daily_nutrients_list.append((meal, meal_nutrients))
        total_day_nutrients['热量 (kcal)'] += meal_nutrients['热量 (kcal)']
        total_day_nutrients['蛋白质 (g)'] += meal_nutrients['蛋白质 (g)']
        total_day_nutrients['脂肪 (g)'] += meal_nutrients['脂肪 (g)']
        total_day_nutrients['碳水化合物 (g)'] += meal_nutrients['碳水化合物 (g)']

    return total_day_nutrients, daily_nutrients_list

# 选择最合理的调整建议并修改膳食
def apply_adjustments(meals, adjustments):
    updated_meals = meals.copy()
    
    # 处理减少部分
    for food, (meal, servings_to_adjust) in adjustments['减少'].items():
        if food in updated_meals[meal]:
            updated_meals[meal].remove(food)
    
    # 处理增加部分
    for food, (meal, servings_to_add) in adjustments['增加'].items():
        if meal in updated_meals:
            updated_meals[meal].append(food)
        else:
            updated_meals[meal] = [food]
    
    return updated_meals

# 标准化膳食数据
def standardize_data(actual_intake, reference_intake):
    standardized_data = {}
    for nutrient in actual_intake:
        standardized_data[nutrient] = actual_intake[nutrient] / reference_intake[nutrient]
    return standardized_data

# 计算灰色关联度
def calculate_grey_relation(actual, reference, rho=0.5):
    delta = {nutrient: abs(actual[nutrient] - reference[nutrient]) for nutrient in actual}
    delta_min = min(delta.values())
    delta_max = max(delta.values())
    grey_relation = {nutrient: (delta_min + rho * delta_max) / (delta[nutrient] + rho * delta_max) for nutrient in delta}
    return grey_relation

# 综合关联度评价
def comprehensive_evaluation(grey_relation):
    return np.mean(list(grey_relation.values()))

# 生成减少和增加食物的所有组合
def generate_combinations(items, max_combinations):
    return list(itertools.combinations(items, max_combinations))

# 定义食物调整策略
def adjust_meals(meals, food_df, nutrients_df, deficiencies, excesses):
    adjustments = {'增加': {}, '减少': {}}
    
    for nutrient in excesses:
        for meal, items in meals.items():
            for item in items:
                food_info = food_df[food_df['食物名称'] == item]
                if not food_info.empty:
                    food_info = food_info.iloc[0]
                    nutrient_info = nutrients_df[nutrients_df['食物'] == food_info['主要成分']].iloc[0]
                    portion_size = food_info['可食部（克/份）'] / 100
                    if nutrient in nutrient_info:
                        reduction_needed = excesses[nutrient]
                        reduction_per_serving = nutrient_info[nutrient] * portion_size
                        if reduction_per_serving > 0:
                            servings_to_reduce = reduction_needed / reduction_per_serving
                            adjustments['减少'][item] = (meal, min(1, servings_to_reduce))
                            excesses[nutrient] -= reduction_per_serving * min(1, servings_to_reduce)
                            if excesses[nutrient] <= 0:
                                break
            if excesses[nutrient] <= 0:
                break

    for nutrient in deficiencies:
        for meal, items in meals.items():
            for _, food_info in food_df.iterrows():
                if food_info['食物名称'] not in items:
                    nutrient_info = nutrients_df[nutrients_df['食物'] == food_info['主要成分']].iloc[0]
                    portion_size = food_info['可食部（克/份）'] / 100
                    if nutrient in nutrient_info:
                        addition_needed = deficiencies[nutrient]
                        addition_per_serving = nutrient_info[nutrient] * portion_size
                        if addition_per_serving > 0:
                            servings_to_add = addition_needed / addition_per_serving
                            adjustments['增加'][food_info['食物名称']] = (meal, min(1, servings_to_add))
                            deficiencies[nutrient] -= addition_per_serving * min(1, servings_to_add)
                            if deficiencies[nutrient] <= 0:
                                break
            if deficiencies[nutrient] <= 0:
                break
    
    return adjustments

# 处理男大学生的膳食调整
male_adjustments = adjust_meals(meals_male, food_male, foods_nutrients, male_deficiencies, male_excesses)

# 处理女大学生的膳食调整
female_adjustments = adjust_meals(meals_female, food_female, foods_nutrients, female_deficiencies, female_excesses)

reduce_items_male = list(male_adjustments['减少'].keys())
reduce_combinations_male = generate_combinations(reduce_items_male, 1)

add_items_female = list(female_adjustments['增加'].keys())
add_combinations_female = generate_combinations(add_items_female, 1)

best_score = -1
best_male_adjustments = None
best_female_adjustments = None
# 遍历所有组合，找到最佳的男生和女生膳食调整方案
for reduce_combination in reduce_combinations_male:
    temp_male_adjustments = {'减少': {k: male_adjustments['减少'][k] for k in reduce_combination}, '增加': {}}
    
    for add_combination in add_combinations_female:
        temp_female_adjustments = {'增加': {k: female_adjustments['增加'][k] for k in add_combination}, '减少': {}}
        
        updated_meals_male = apply_adjustments(meals_male, temp_male_adjustments)
        updated_meals_female = apply_adjustments(meals_female, temp_female_adjustments)
        
        updated_nutrients_male, _ = calculate_nutrients(updated_meals_male, food_male)
        updated_nutrients_female, _ = calculate_nutrients(updated_meals_female, food_female)
        
        standardized_nutrients_male = standardize_data(updated_nutrients_male, reference_intake_male)
        standardized_nutrients_female = standardize_data(updated_nutrients_female, reference_intake_female)
        
        grey_relation_male = calculate_grey_relation(standardized_nutrients_male, reference_intake_male)
        grey_relation_female = calculate_grey_relation(standardized_nutrients_female, reference_intake_female)
        
        score_male = comprehensive_evaluation(grey_relation_male)
        score_female = comprehensive_evaluation(grey_relation_female)
        
        total_score = score_male + score_female
        
        if total_score > best_score:
            best_score = total_score
            best_male_adjustments = temp_male_adjustments
            best_female_adjustments = temp_female_adjustments

updated_meals_male = apply_adjustments(meals_male, best_male_adjustments)
updated_meals_female = apply_adjustments(meals_female, best_female_adjustments)

updated_whole_day_nutrients_male, updated_male_ls = calculate_nutrients(updated_meals_male, food_male)
updated_whole_day_nutrients_female, updated_female_ls = calculate_nutrients(updated_meals_female, food_female)

updated_male_standardized = standardize_data(updated_whole_day_nutrients_male, reference_intake_male)
updated_female_standardized = standardize_data(updated_whole_day_nutrients_female, reference_intake_female)

updated_male_grey_relation = calculate_grey_relation(updated_male_standardized, reference_intake_male)
updated_female_grey_relation = calculate_grey_relation(updated_female_standardized, reference_intake_female)

updated_male_comprehensive_score = comprehensive_evaluation(updated_male_grey_relation)
updated_female_comprehensive_score = comprehensive_evaluation(updated_female_grey_relation)

print(f"更新后男大学生膳食的综合灰色关联度评分：{updated_male_comprehensive_score}")
print(f"更新后女大学生膳食的综合灰色关联度评分：{updated_female_comprehensive_score}")

print("\n男大学生膳食的调整建议：")
for action, changes in best_male_adjustments.items():
    for food, (meal, servings) in changes.items():
        print(f"{action} {servings} 份 {food} 在 {meal}")

print("\n女大学生膳食的调整建议：")
for action, changes in best_female_adjustments.items():
    for food, (meal, servings) in changes.items():
        print(f"{action} {servings} 份 {food} 在 {meal}")


更新后男大学生膳食的综合灰色关联度评分：0.7954812715039575
更新后女大学生膳食的综合灰色关联度评分：0.7927958512083574

男大学生膳食的调整建议：
减少 1 份 小米粥 在 早餐

女大学生膳食的调整建议：
增加 1 份 鸡蛋饼 在 早餐
